## Set up the dependencies

In [1]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev
import arrow

In [2]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev

In [3]:
# For plots
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# For maps
import folium
import branca.element as bre

In [5]:
# For easier debugging while working on modules
import importlib

In [6]:
import pandas as pd
import numpy as np

## The spec

The spec defines what experiments were done, and over which time ranges. Once the experiment is complete, most of the structure is read back from the data, but we use the spec to validate that it all worked correctly. The spec also contains the ground truth for the legs. Here, we read the spec for the trip to UC Berkeley.

In [7]:
DATASTORE_LOC = "bin/data/"
AUTHOR_EMAIL = "shankari@eecs.berkeley.edu"
sd_la = eisd.FileSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, "unimodal_trip_car_bike_mtv_la")
sd_sj = eisd.FileSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, "car_scooter_brex_san_jose")
sd_ucb = eisd.FileSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, "train_bus_ebike_mtv_ucb")

About to retrieve messages using {'user': 'shankari@eecs.berkeley.edu', 'key_list': ['config/evaluation_spec'], 'start_time': 0, 'end_time': 1613685967}
response = <Response [200]>
Found 20 entries
After iterating over 20 entries, entry found
Found spec = Round trip car and bike trip in the South Bay
Evaluation ran from 2019-07-20T00:00:00-07:00 -> 2020-04-29T17:00:00-07:00
About to retrieve messages using {'user': 'shankari@eecs.berkeley.edu', 'key_list': ['config/evaluation_spec'], 'start_time': 0, 'end_time': 1613685968}
response = <Response [200]>
Found 20 entries
After iterating over 20 entries, entry found
Found spec = Multi-modal car scooter BREX trip to San Jose
Evaluation ran from 2019-07-20T00:00:00-07:00 -> 2020-04-29T17:00:00-07:00
About to retrieve messages using {'user': 'shankari@eecs.berkeley.edu', 'key_list': ['config/evaluation_spec'], 'start_time': 0, 'end_time': 1613685969}
response = <Response [200]>
Found 20 entries
After iterating over 20 entries, entry found
Fou

## Loading the data into a dataframe

In [8]:
pv_la = eipv.PhoneView(sd_la)

-------------------- About to read transitions from server --------------------
Reading data for android phones
Loading transitions for phone ucb-sdb-android-1
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['manual/evaluation_transition'], 'start_time': 1563606000, 'end_time': 1588204800}
response = <Response [200]>
Found 86 entries
Loading transitions for phone ucb-sdb-android-2
About to retrieve messages using {'user': 'ucb-sdb-android-2', 'key_list': ['manual/evaluation_transition'], 'start_time': 1563606000, 'end_time': 1588204800}
response = <Response [200]>
Found 86 entries
Loading transitions for phone ucb-sdb-android-3
About to retrieve messages using {'user': 'ucb-sdb-android-3', 'key_list': ['manual/evaluation_transition'], 'start_time': 1563606000, 'end_time': 1588204800}
response = <Response [200]>
Found 86 entries
Loading transitions for phone ucb-sdb-android-4
About to retrieve messages using {'user': 'ucb-sdb-android-4', 'key_list': ['manual/

response = <Response [200]>
Found 7 entries
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['background/battery'], 'start_time': 1564334117.295, 'end_time': 1564343045.9}
response = <Response [200]>
Found 11 entries
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['background/battery'], 'start_time': 1564351305.633, 'end_time': 1564360156.392}
response = <Response [200]>
Found 9 entries
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['background/battery'], 'start_time': 1565571034.93, 'end_time': 1565579009.992}
response = <Response [200]>
Found 11 entries
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['background/battery'], 'start_time': 1567271203.764, 'end_time': 1567279427.961}
response = <Response [200]>
Found 9 entries
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['background/battery'], 'start_time': 1567288614.642, 'end_time': 1567297340.9

response = <Response [200]>
Found 11 entries
About to retrieve messages using {'user': 'ucb-sdb-ios-4', 'key_list': ['background/battery'], 'start_time': 1564334069.1049929, 'end_time': 1564343007.721693}
response = <Response [200]>
Found 11 entries
About to retrieve messages using {'user': 'ucb-sdb-ios-4', 'key_list': ['background/battery'], 'start_time': 1564351193.2504249, 'end_time': 1564360033.887364}
response = <Response [200]>
Found 12 entries
About to retrieve messages using {'user': 'ucb-sdb-ios-4', 'key_list': ['background/battery'], 'start_time': 1565570989.744656, 'end_time': 1565578915.798939}
response = <Response [200]>
Found 13 entries
About to retrieve messages using {'user': 'ucb-sdb-ios-4', 'key_list': ['background/battery'], 'start_time': 1567271164.779111, 'end_time': 1567279350.7691588}
response = <Response [200]>
Found 11 entries
About to retrieve messages using {'user': 'ucb-sdb-ios-4', 'key_list': ['background/battery'], 'start_time': 1567288593.3784451, 'end_ti

response = <Response [200]>
Found 1868 entries
Retrieved 1868 entries with timestamps [1564274558.354, 1564274558.727, 1564274564.028, 1564274565, 1564274566, 1564274567, 1564274568, 1564274569, 1564274570, 1564274571]...
About to retrieve data for ucb-sdb-android-2 from 1564280671.731 -> 1564282402.886
About to retrieve messages using {'user': 'ucb-sdb-android-2', 'key_list': ['background/location'], 'start_time': 1564280671.731, 'end_time': 1564282402.886}
response = <Response [200]>
Found 1 entries
Retrieved 1 entries with timestamps [1564280652]...
About to retrieve data for ucb-sdb-android-2 from 1564274304.968 -> 1564282402.886
About to retrieve messages using {'user': 'ucb-sdb-android-2', 'key_list': ['background/filtered_location'], 'start_time': 1564274304.968, 'end_time': 1564282402.886}
response = <Response [200]>
Found 1854 entries
Retrieved 1854 entries with timestamps [1564274558.727, 1564274565, 1564274566, 1564274567, 1564274568, 1564274569, 1564274570, 1564274571, 1564

response = <Response [200]>
Found 2121 entries
Retrieved 2121 entries with timestamps [1567288912, 1567288913, 1567288914, 1567288915, 1567288916, 1567288917, 1567288918, 1567288919, 1567288920, 1567288921]...
About to retrieve data for ucb-sdb-android-2 from 1567296306.499 -> 1567297357.197
About to retrieve messages using {'user': 'ucb-sdb-android-2', 'key_list': ['background/filtered_location'], 'start_time': 1567296306.499, 'end_time': 1567297357.197}
response = <Response [200]>
Found 1 entries
Retrieved 1 entries with timestamps [1567296305.357]...
About to retrieve data for ucb-sdb-android-3 from 1564274288.319 -> 1564282423.072
About to retrieve messages using {'user': 'ucb-sdb-android-3', 'key_list': ['background/location'], 'start_time': 1564274288.319, 'end_time': 1564282423.072}
response = <Response [200]>
Found 95 entries
Retrieved 95 entries with timestamps [1564274778.389, 1564274778.389, 1564274795, 1564274812.987, 1564274819, 1564274844.433, 1564274875, 1564274907.723, 

...

response = <Response [200]>
Found 437 entries
Retrieved 437 entries with timestamps [1565571245.513, 1565571255.862, 1565571272.654, 1565571284.949, 1565571290.953, 1565571296.96, 1565571297.455, 1565571308.955, 1565571309.468, 1565571315.43]...
About to retrieve data for ucb-sdb-android-3 from 1565577786.12 -> 1565578932.123
About to retrieve messages using {'user': 'ucb-sdb-android-3', 'key_list': ['background/motion_activity'], 'start_time': 1565577786.12, 'end_time': 1565578932.123}
response = <Response [200]>
Found 1 entries
Retrieved 1 entries with timestamps [1565577786.12]...
motion activity has not been processed, copying write_ts -> ts
About to retrieve data for ucb-sdb-android-3 from 1567271178.821 -> 1567279372.922
About to retrieve messages using {'user': 'ucb-sdb-android-3', 'key_list': ['background/motion_activity'], 'start_time': 1567271178.821, 'end_time': 1567279372.922}
response = <Response [200]>
Found 326 entries
Retrieved 326 entries with timestamps [1567271522.47

response = <Response [200]>
Found 472 entries
Retrieved 472 entries with timestamps [1564351283.6382508, 1564351323.8419127, 1564351326.3507926, 1564351471.793641, 1564351564.5028896, 1564351577.031905, 1564351597.0661352, 1564351617.101654, 1564351629.6283417, 1564351682.2153893]...
About to retrieve data for ucb-sdb-ios-1 from 1564359606.3683307 -> 1564359997.0496612
About to retrieve messages using {'user': 'ucb-sdb-ios-1', 'key_list': ['background/motion_activity'], 'start_time': 1564359606.3683307, 'end_time': 1564359997.0496612}
response = <Response [200]>
Found 1 entries
Retrieved 1 entries with timestamps [1564359606.3683307]...
About to retrieve data for ucb-sdb-ios-1 from 1565570971.32563 -> 1565578881.489321
About to retrieve messages using {'user': 'ucb-sdb-ios-1', 'key_list': ['background/motion_activity'], 'start_time': 1565570971.32563, 'end_time': 1565578881.489321}
response = <Response [200]>
Found 470 entries
Retrieved 470 entries with timestamps [1565571015.1305943, 

response = <Response [200]>
Found 225 entries
Retrieved 225 entries with timestamps [1564274317.477725, 1564274347.4777246, 1564274437.477725, 1564274542.4777246, 1564274552.4777243, 1564274607.709919, 1564274615.3459363, 1564274648.6975765, 1564274650.9377193, 1564274653.479989]...
About to retrieve data for ucb-sdb-ios-3 from 1564282309.445576 -> 1564282322.549677
About to retrieve messages using {'user': 'ucb-sdb-ios-3', 'key_list': ['background/motion_activity'], 'start_time': 1564282309.445576, 'end_time': 1564282322.549677}
response = <Response [200]>
Found 1 entries
Retrieved 1 entries with timestamps [1564282309.445576]...
About to retrieve data for ucb-sdb-ios-3 from 1564334084.0051398 -> 1564342996.99437
About to retrieve messages using {'user': 'ucb-sdb-ios-3', 'key_list': ['background/motion_activity'], 'start_time': 1564334084.0051398, 'end_time': 1564342996.99437}
response = <Response [200]>
Found 180 entries
Retrieved 180 entries with timestamps [1564334169.4373546, 1564

response = <Response [200]>
Found 151 entries
Retrieved 151 entries with timestamps [1567271272.0296657, 1567271428.441823, 1567271498.4418228, 1567271523.441823, 1567271533.4418225, 1567271543.4418225, 1567271553.441823, 1567271758.2811177, 1567272125.8327036, 1567272295.832704]...
About to retrieve data for ucb-sdb-ios-4 from 1567279350.54006 -> 1567279350.7691588
About to retrieve messages using {'user': 'ucb-sdb-ios-4', 'key_list': ['background/motion_activity'], 'start_time': 1567279350.54006, 'end_time': 1567279350.7691588}
response = <Response [200]>
Found 1 entries
Retrieved 1 entries with timestamps [1567279350.54006]...
About to retrieve data for ucb-sdb-ios-4 from 1567288593.3784451 -> 1567297325.508904
About to retrieve messages using {'user': 'ucb-sdb-ios-4', 'key_list': ['background/motion_activity'], 'start_time': 1567288593.3784451, 'end_time': 1567297325.508904}
response = <Response [200]>
Found 224 entries
Retrieved 224 entries with timestamps [1567288679.663621, 1567

response = <Response [200]>
Found 18 entries
transition has not been processed, creating ts -> fmt_time
About to retrieve messages using {'user': 'ucb-sdb-ios-1', 'key_list': ['statemachine/transition'], 'start_time': 1564333996.5363011, 'end_time': 1564342975.472409}
response = <Response [200]>
Found 15 entries
transition has not been processed, creating ts -> fmt_time
About to retrieve messages using {'user': 'ucb-sdb-ios-1', 'key_list': ['statemachine/transition'], 'start_time': 1564351246.07085, 'end_time': 1564359997.0496612}
response = <Response [200]>
Found 17 entries
transition has not been processed, creating ts -> fmt_time
About to retrieve messages using {'user': 'ucb-sdb-ios-1', 'key_list': ['statemachine/transition'], 'start_time': 1565570971.32563, 'end_time': 1565578881.489321}
response = <Response [200]>
Found 17 entries
transition has not been processed, creating ts -> fmt_time
About to retrieve messages using {'user': 'ucb-sdb-ios-1', 'key_list': ['statemachine/transi

In [9]:
pv_sj = eipv.PhoneView(sd_sj)

-------------------- About to read transitions from server --------------------
Reading data for android phones
Loading transitions for phone ucb-sdb-android-1
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['manual/evaluation_transition'], 'start_time': 1563606000, 'end_time': 1588204800}
response = <Response [200]>
Found 86 entries
Loading transitions for phone ucb-sdb-android-2
About to retrieve messages using {'user': 'ucb-sdb-android-2', 'key_list': ['manual/evaluation_transition'], 'start_time': 1563606000, 'end_time': 1588204800}
response = <Response [200]>
Found 86 entries
Loading transitions for phone ucb-sdb-android-3
About to retrieve messages using {'user': 'ucb-sdb-android-3', 'key_list': ['manual/evaluation_transition'], 'start_time': 1563606000, 'end_time': 1588204800}
response = <Response [200]>
Found 86 entries
Loading transitions for phone ucb-sdb-android-4
About to retrieve messages using {'user': 'ucb-sdb-android-4', 'key_list': ['manual/

response = <Response [200]>
Found 9 entries
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['background/battery'], 'start_time': 1563896743.139, 'end_time': 1563917431.489}
response = <Response [200]>
Found 11 entries
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['background/battery'], 'start_time': 1564244087.215, 'end_time': 1564263878.59}
response = <Response [200]>
Found 10 entries
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['background/battery'], 'start_time': 1565026006.793, 'end_time': 1565050217.383}
response = <Response [200]>
Found 12 entries
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['background/battery'], 'start_time': 1565116007.631, 'end_time': 1565140213.84}
response = <Response [200]>
Found 11 entries
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['background/battery'], 'start_time': 1565194658.17, 'end_time': 1565217816

response = <Response [200]>
Found 29 entries
About to retrieve messages using {'user': 'ucb-sdb-ios-3', 'key_list': ['background/battery'], 'start_time': 1563821405.729749, 'end_time': 1563846186.997916}
response = <Response [200]>
Found 29 entries
About to retrieve messages using {'user': 'ucb-sdb-ios-3', 'key_list': ['background/battery'], 'start_time': 1563896712.148884, 'end_time': 1563917670.784336}
response = <Response [200]>
Found 36 entries
About to retrieve messages using {'user': 'ucb-sdb-ios-3', 'key_list': ['background/battery'], 'start_time': 1564244052.238229, 'end_time': 1564263855.954498}
response = <Response [200]>
Found 51 entries
About to retrieve messages using {'user': 'ucb-sdb-ios-3', 'key_list': ['background/battery'], 'start_time': 1565025989.940984, 'end_time': 1565050162.005754}
response = <Response [200]>
Found 50 entries
About to retrieve messages using {'user': 'ucb-sdb-ios-3', 'key_list': ['background/battery'], 'start_time': 1565115954.616665, 'end_time':

About to retrieve data for ucb-sdb-android-1 from 1565194658.17 -> 1565217816.765
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['background/location'], 'start_time': 1565194658.17, 'end_time': 1565217816.765}
response = <Response [200]>
Found 9119 entries
Retrieved 9119 entries with timestamps [1565194658.375, 1565194664, 1565194665, 1565194666, 1565194667, 1565194668, 1565194669, 1565194670, 1565194671, 1565194672]...
About to retrieve data for ucb-sdb-android-1 from 1565217816.547 -> 1565217816.765
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['background/location'], 'start_time': 1565217816.547, 'end_time': 1565217816.765}
response = <Response [200]>
Found 1 entries
Retrieved 1 entries with timestamps [1565217817]...
About to retrieve data for ucb-sdb-android-1 from 1565194658.17 -> 1565217816.765
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['background/filtered_location'], 'start_time': 15651

response = <Response [200]>
Found 6304 entries
Retrieved 6304 entries with timestamps [1565116261.81, 1565116323.781, 1565116328.938, 1565116334.023, 1565116339.103, 1565116345, 1565116346, 1565116347, 1565116348, 1565116349]...
About to retrieve data for ucb-sdb-android-2 from 1565137655.504 -> 1565140246.406
About to retrieve messages using {'user': 'ucb-sdb-android-2', 'key_list': ['background/location'], 'start_time': 1565137655.504, 'end_time': 1565140246.406}
response = <Response [200]>
Found 1 entries
Retrieved 1 entries with timestamps [1565137654.665]...
About to retrieve data for ucb-sdb-android-2 from 1565116024.103 -> 1565140246.406
About to retrieve messages using {'user': 'ucb-sdb-android-2', 'key_list': ['background/filtered_location'], 'start_time': 1565116024.103, 'end_time': 1565140246.406}
response = <Response [200]>
Found 6237 entries
Retrieved 6237 entries with timestamps [1565116323.781, 1565116328.938, 1565116334.023, 1565116339.103, 1565116345, 1565116346, 15651

...

response = <Response [200]>
Found 407 entries
Retrieved 407 entries with timestamps [1565194758.0400949, 1565194774.885702, 1565194865.3019307, 1565194875.350277, 1565194877.86166, 1565194925.5680397, 1565194928.0789037, 1565194954.7441468, 1565194963.2201753, 1565195061.1372747]...
About to retrieve data for ucb-sdb-ios-1 from 1565217701.1632712 -> 1565217732.972708
About to retrieve messages using {'user': 'ucb-sdb-ios-1', 'key_list': ['background/motion_activity'], 'start_time': 1565217701.1632712, 'end_time': 1565217732.972708}
response = <Response [200]>
Found 1 entries
Retrieved 1 entries with timestamps [1565217701.1632712]...
About to retrieve data for ucb-sdb-ios-1 from 1583349504.692752 -> 1583372462.538796
About to retrieve messages using {'user': 'ucb-sdb-ios-1', 'key_list': ['background/motion_activity'], 'start_time': 1583349504.692752, 'end_time': 1583372462.538796}
response = <Response [200]>
Found 511 entries
Retrieved 511 entries with timestamps [1583349517.4804544, 1

response = <Response [200]>
Found 328 entries
Retrieved 328 entries with timestamps [1563896727.4501264, 1563896729.3657155, 1563896746.744403, 1563896756.7444026, 1563896816.7444034, 1563896821.7444031, 1563896881.744403, 1563896891.7444026, 1563896931.7444036, 1563897016.028275]...
About to retrieve data for ucb-sdb-ios-3 from 1563916932.3446546 -> 1563917670.784336
About to retrieve messages using {'user': 'ucb-sdb-ios-3', 'key_list': ['background/motion_activity'], 'start_time': 1563916932.3446546, 'end_time': 1563917670.784336}
response = <Response [200]>
Found 1 entries
Retrieved 1 entries with timestamps [1563916932.3446546]...
About to retrieve data for ucb-sdb-ios-3 from 1564244052.238229 -> 1564263855.954498
About to retrieve messages using {'user': 'ucb-sdb-ios-3', 'key_list': ['background/motion_activity'], 'start_time': 1564244052.238229, 'end_time': 1564263855.954498}
response = <Response [200]>
Found 422 entries
Retrieved 422 entries with timestamps [1564244125.9491072, 

response = <Response [200]>
Found 309 entries
Retrieved 309 entries with timestamps [1565115984.4551802, 1565116037.2279844, 1565116052.227984, 1565116057.227984, 1565116102.2279837, 1565116177.2279835, 1565116187.227984, 1565116227.2279837, 1565116322.2279844, 1565116327.2279844]...
About to retrieve data for ucb-sdb-ios-4 from 1565140133.077963 -> 1565140156.261787
About to retrieve messages using {'user': 'ucb-sdb-ios-4', 'key_list': ['background/motion_activity'], 'start_time': 1565140133.077963, 'end_time': 1565140156.261787}
response = <Response [200]>
Found 1 entries
Retrieved 1 entries with timestamps [1565140133.077963]...
About to retrieve data for ucb-sdb-ios-4 from 1565194696.183675 -> 1565217805.7350268
About to retrieve messages using {'user': 'ucb-sdb-ios-4', 'key_list': ['background/motion_activity'], 'start_time': 1565194696.183675, 'end_time': 1565217805.7350268}
response = <Response [200]>
Found 200 entries
Retrieved 200 entries with timestamps [1565194751.5046077, 1

response = <Response [200]>
Found 3 entries
transition has not been processed, creating ts -> fmt_time
About to retrieve messages using {'user': 'ucb-sdb-android-4', 'key_list': ['statemachine/transition'], 'start_time': 1564244116.137, 'end_time': 1564263954.233}
response = <Response [200]>
Found 1 entries
transition has not been processed, creating ts -> fmt_time
About to retrieve messages using {'user': 'ucb-sdb-android-4', 'key_list': ['statemachine/transition'], 'start_time': 1565026039.443, 'end_time': 1565050297.499}
response = <Response [200]>
Found 1 entries
transition has not been processed, creating ts -> fmt_time
About to retrieve messages using {'user': 'ucb-sdb-android-4', 'key_list': ['statemachine/transition'], 'start_time': 1565116073.834, 'end_time': 1565140325.815}
response = <Response [200]>
Found 1 entries
transition has not been processed, creating ts -> fmt_time
About to retrieve messages using {'user': 'ucb-sdb-android-4', 'key_list': ['statemachine/transition']

response = <Response [200]>
Found 6 entries
transition has not been processed, creating ts -> fmt_time
-------------------- About to select trip specific ranges --------------------
Filling label ucb-sdb-android-1 for OS android
Filling label ucb-sdb-android-2 for OS android
Filling label ucb-sdb-android-3 for OS android
Filling label ucb-sdb-android-4 for OS android
Filling label ucb-sdb-ios-1 for OS ios
Filling label ucb-sdb-ios-2 for OS ios
Filling label ucb-sdb-ios-3 for OS ios
Filling label ucb-sdb-ios-4 for OS ios
-------------------- Done populating information from server --------------------


In [10]:
sd_sj.CURR_SPEC_ID

'car_scooter_brex_san_jose'

In [11]:
ios_loc_entries = sd_sj.retrieve_data("ucb-sdb-ios-1", ["background/location"],
                                arrow.get("2019-08-07T14:50:57.445000-07:00").timestamp,
                                arrow.get("2019-08-07T15:00:16.787000-07:00").timestamp)
ios_location_df = pd.DataFrame([e["data"] for e in ios_loc_entries])

About to retrieve messages using {'user': 'ucb-sdb-ios-1', 'key_list': ['background/location'], 'start_time': 1565214657, 'end_time': 1565215216}
response = <Response [200]>
Found 80 entries


In [12]:
android_loc_entries = sd_sj.retrieve_data("ucb-sdb-android-1", ["background/location"],
                                arrow.get("2019-08-07T14:50:57.445000-07:00").timestamp,
                                arrow.get("2019-08-07T15:00:16.787000-07:00").timestamp)
android_location_df = pd.DataFrame([e["data"] for e in android_loc_entries])

About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['background/location'], 'start_time': 1565214657, 'end_time': 1565215216}
response = <Response [200]>
Found 146 entries


In [13]:
android_location_df[["fmt_time"]].loc[30:60]

,fmt_time
30,"Aug 7, 2019 2:51:28 PM"
31,"Aug 7, 2019 2:51:29 PM"
32,"Aug 7, 2019 2:51:30 PM"
33,"Aug 7, 2019 2:51:31 PM"
34,"Aug 7, 2019 2:51:32 PM"
35,"Aug 7, 2019 2:51:33 PM"
36,"Aug 7, 2019 2:51:34 PM"
37,"Aug 7, 2019 2:51:35 PM"
38,"Aug 7, 2019 2:51:36 PM"
39,"Aug 7, 2019 2:51:37 PM"


In [14]:
ios_map = ezpv.display_map_detail_from_df(ios_location_df.loc[20:35])
android_map = ezpv.display_map_detail_from_df(android_location_df.loc[25:50])

In [15]:
fig = bre.Figure()
fig.add_subplot(1, 2, 1).add_child(ios_map)
fig.add_subplot(1, 2, 2).add_child(android_map)

In [16]:
pv_ucb = eipv.PhoneView(sd_ucb)

-------------------- About to read transitions from server --------------------
Reading data for android phones
Loading transitions for phone ucb-sdb-android-1
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['manual/evaluation_transition'], 'start_time': 1563260400, 'end_time': 1567148400}
response = <Response [200]>
Found 72 entries
Loading transitions for phone ucb-sdb-android-2
About to retrieve messages using {'user': 'ucb-sdb-android-2', 'key_list': ['manual/evaluation_transition'], 'start_time': 1563260400, 'end_time': 1567148400}
response = <Response [200]>
Found 60 entries
Loading transitions for phone ucb-sdb-android-3
About to retrieve messages using {'user': 'ucb-sdb-android-3', 'key_list': ['manual/evaluation_transition'], 'start_time': 1563260400, 'end_time': 1567148400}
response = <Response [200]>
Found 60 entries
Loading transitions for phone ucb-sdb-android-4
About to retrieve messages using {'user': 'ucb-sdb-android-4', 'key_list': ['manual/

response = <Response [200]>
Found 24 entries
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['background/battery'], 'start_time': 1564067115.279, 'end_time': 1564111993.08}
response = <Response [200]>
Found 20 entries
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['background/battery'], 'start_time': 1564153794.979, 'end_time': 1564198662.867}
response = <Response [200]>
Found 22 entries
About to retrieve messages using {'user': 'ucb-sdb-android-2', 'key_list': ['background/battery'], 'start_time': 1563979684.205, 'end_time': 1564026016.571}
response = <Response [200]>
Found 37 entries
About to retrieve messages using {'user': 'ucb-sdb-android-2', 'key_list': ['background/battery'], 'start_time': 1564067129.194, 'end_time': 1564111993.08}
response = <Response [200]>
Found 30 entries
About to retrieve messages using {'user': 'ucb-sdb-android-2', 'key_list': ['background/battery'], 'start_time': 1564153809.884, 'end_time': 15641986

response = <Response [200]>
Found 0 entries
Retrieved 0 entries with timestamps []...
About to retrieve data for ucb-sdb-android-2 from 1563979684.205 -> 1564026016.571
About to retrieve messages using {'user': 'ucb-sdb-android-2', 'key_list': ['background/location'], 'start_time': 1563979684.205, 'end_time': 1564026016.571}
response = <Response [200]>
Found 14149 entries
Retrieved 14149 entries with timestamps [1563979683.523, 1563979683.523, 1563979686.541, 1563979691, 1563979692, 1563979693, 1563979694, 1563979695, 1563979696, 1563979697]...
About to retrieve data for ucb-sdb-android-2 from 1564021868.503 -> 1564026016.571
About to retrieve messages using {'user': 'ucb-sdb-android-2', 'key_list': ['background/location'], 'start_time': 1564021868.503, 'end_time': 1564026016.571}
response = <Response [200]>
Found 1 entries
Retrieved 1 entries with timestamps [1564021794]...
About to retrieve data for ucb-sdb-android-2 from 1563979684.205 -> 1564026016.571
About to retrieve messages us

response = <Response [200]>
Found 875 entries
Retrieved 875 entries with timestamps [1564154023.987, 1564154023.987, 1564154050, 1564154055, 1564154088.096, 1564154119.23, 1564154150, 1564154180, 1564154211, 1564154241]...
About to retrieve data for ucb-sdb-android-3 from 1564196589.192 -> 1564198700.606
About to retrieve messages using {'user': 'ucb-sdb-android-3', 'key_list': ['background/location'], 'start_time': 1564196589.192, 'end_time': 1564198700.606}
response = <Response [200]>
Found 1 entries
Retrieved 1 entries with timestamps [1564196587.808]...
About to retrieve data for ucb-sdb-android-3 from 1564153836.582 -> 1564198700.606
About to retrieve messages using {'user': 'ucb-sdb-android-3', 'key_list': ['background/filtered_location'], 'start_time': 1564153836.582, 'end_time': 1564198700.606}
response = <Response [200]>
Found 721 entries
Retrieved 721 entries with timestamps [1564154023.987, 1564154050, 1564154055, 1564154088.096, 1564154119.23, 1564154150, 1564154180, 156415

response = <Response [200]>
Found 11052 entries
Retrieved 11052 entries with timestamps [1563980042.8723888, 1563980044.385489, 1563980046.1162372, 1563980047.1293101, 1563980047.999834, 1563980048.9998174, 1563980049.9998074, 1563980050.9997978, 1563980051.9997861, 1563980053.999772]...
About to retrieve data for ucb-sdb-ios-2 from 1564023575.128773 -> 1564026158.946105
About to retrieve messages using {'user': 'ucb-sdb-ios-2', 'key_list': ['background/location'], 'start_time': 1564023575.128773, 'end_time': 1564026158.946105}
response = <Response [200]>
Found 1 entries
Retrieved 1 entries with timestamps [1564023574.9863236]...
About to retrieve data for ucb-sdb-ios-2 from 1563979628.6823301 -> 1564026158.946105
About to retrieve messages using {'user': 'ucb-sdb-ios-2', 'key_list': ['background/filtered_location'], 'start_time': 1563979628.6823301, 'end_time': 1564026158.946105}
response = <Response [200]>
Found 10525 entries
Retrieved 10525 entries with timestamps [1563980044.385489

response = <Response [200]>
Found 2321 entries
Retrieved 2321 entries with timestamps [1564117209.9645019, 1564154572.964088, 1564154849.341601, 1564155206.677164, 1564155222.038831, 1564155229.005299, 1564155244.811119, 1564155251.220082, 1564155273.199754, 1564155282.301773]...
About to retrieve data for ucb-sdb-ios-3 from 1564198595.263564 -> 1564198601.973415
About to retrieve messages using {'user': 'ucb-sdb-ios-3', 'key_list': ['background/location'], 'start_time': 1564198595.263564, 'end_time': 1564198601.973415}
response = <Response [200]>
Found 1 entries
Retrieved 1 entries with timestamps [1564198595.1684551]...
About to retrieve data for ucb-sdb-ios-3 from 1564153754.09529 -> 1564198601.973415
About to retrieve messages using {'user': 'ucb-sdb-ios-3', 'key_list': ['background/filtered_location'], 'start_time': 1564153754.09529, 'end_time': 1564198601.973415}
response = <Response [200]>
Found 1858 entries
Retrieved 1858 entries with timestamps [1564154572.964088, 1564154849.3

response = <Response [200]>
Found 1587 entries
Retrieved 1587 entries with timestamps [1564154036.632, 1564154049, 1564154057.113, 1564154065.3, 1564154078.39, 1564154091.102, 1564154103.719, 1564154116.522, 1564154125.219, 1564154134.269]...
About to retrieve data for ucb-sdb-android-2 from 1564193889.939 -> 1564198662.867
About to retrieve messages using {'user': 'ucb-sdb-android-2', 'key_list': ['background/motion_activity'], 'start_time': 1564193889.939, 'end_time': 1564198662.867}
response = <Response [200]>
Found 1 entries
Retrieved 1 entries with timestamps [1564193889.939]...
motion activity has not been processed, copying write_ts -> ts
About to retrieve data for ucb-sdb-android-3 from 1563979700.651 -> 1564026069.823
About to retrieve messages using {'user': 'ucb-sdb-android-3', 'key_list': ['background/motion_activity'], 'start_time': 1563979700.651, 'end_time': 1564026069.823}
response = <Response [200]>
Found 698 entries
Retrieved 698 entries with timestamps [1563980015.46

response = <Response [200]>
Found 927 entries
Retrieved 927 entries with timestamps [1563979677.622645, 1563979767.6226444, 1563979772.6226444, 1563979787.6226444, 1563980990.0595727, 1563980993.8012114, 1563981015.9809394, 1563981085.6387854, 1563981089.7018366, 1563981093.760253]...
About to retrieve data for ucb-sdb-ios-2 from 1564026145.1770449 -> 1564026158.946105
About to retrieve messages using {'user': 'ucb-sdb-ios-2', 'key_list': ['background/motion_activity'], 'start_time': 1564026145.1770449, 'end_time': 1564026158.946105}
response = <Response [200]>
Found 1 entries
Retrieved 1 entries with timestamps [1564026145.1770449]...
About to retrieve data for ucb-sdb-ios-2 from 1564067067.1988769 -> 1564111964.549928
About to retrieve messages using {'user': 'ucb-sdb-ios-2', 'key_list': ['background/motion_activity'], 'start_time': 1564067067.1988769, 'end_time': 1564111964.549928}
response = <Response [200]>
Found 876 entries
Retrieved 876 entries with timestamps [1564067075.706058

response = <Response [200]>
Found 2 entries
transition has not been processed, creating ts -> fmt_time
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['statemachine/transition'], 'start_time': 1564153794.979, 'end_time': 1564198662.867}
response = <Response [200]>
Found 2 entries
transition has not been processed, creating ts -> fmt_time
About to retrieve messages using {'user': 'ucb-sdb-android-2', 'key_list': ['statemachine/transition'], 'start_time': 1563979684.205, 'end_time': 1564026016.571}
response = <Response [200]>
Found 21 entries
transition has not been processed, creating ts -> fmt_time
About to retrieve messages using {'user': 'ucb-sdb-android-2', 'key_list': ['statemachine/transition'], 'start_time': 1564067129.194, 'end_time': 1564111993.08}
response = <Response [200]>
Found 17 entries
transition has not been processed, creating ts -> fmt_time
About to retrieve messages using {'user': 'ucb-sdb-android-2', 'key_list': ['statemachine/transition'

In [17]:
import pandas as pd

In [18]:
def get_battery_drain_entries(pv):
    battery_entry_list = []
    for phone_os, phone_map in pv.map().items():
        print(15 * "=*")
        print(phone_os, phone_map.keys())
        for phone_label, phone_detail_map in phone_map.items():
            print(4 * ' ', 15 * "-*")
            print(4 * ' ', phone_label, phone_detail_map.keys())
            # this spec does not have any calibration ranges, but evaluation ranges are actually cooler
            for r in phone_detail_map["evaluation_ranges"]:
                print(8 * ' ', 30 * "=")
                print(8 * ' ',r.keys())
                print(8 * ' ',r["trip_id"], r["eval_common_trip_id"], r["eval_role"], len(r["evaluation_trip_ranges"]))
                bcs = r["battery_df"]["battery_level_pct"]
                delta_battery = bcs.iloc[0] - bcs.iloc[-1]
                print("Battery starts at %d, ends at %d, drain = %d" % (bcs.iloc[0], bcs.iloc[-1], delta_battery))
                battery_entry = {"phone_os": phone_os, "phone_label": phone_label, "timeline": pv.spec_details.curr_spec["id"],
                                 "run": r["trip_run"], "duration": r["duration"],
                                 "role": r["eval_role_base"], "battery_drain": delta_battery}
                battery_entry_list.append(battery_entry)
    return battery_entry_list

In [19]:
# We are not going to look at battery life at the evaluation trip level; we will end with evaluation range
# since we want to capture the overall drain for the timeline
battery_entries_list = []
battery_entries_list.extend(get_battery_drain_entries(pv_la))
battery_entries_list.extend(get_battery_drain_entries(pv_sj))
battery_entries_list.extend(get_battery_drain_entries(pv_ucb))
battery_drain_df = pd.DataFrame(battery_entries_list)

=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*
android dict_keys(['ucb-sdb-android-1', 'ucb-sdb-android-2', 'ucb-sdb-android-3', 'ucb-sdb-android-4'])
     -*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
     ucb-sdb-android-1 dict_keys(['role', 'transitions', 'calibration_transitions', 'calibration_ranges', 'evaluation_transitions', 'evaluation_ranges'])
         dict_keys(['trip_id', 'trip_id_base', 'trip_run', 'start_ts', 'end_ts', 'duration', 'eval_common_trip_id', 'eval_role', 'eval_role_base', 'eval_role_run', 'evaluation_trip_ranges', 'battery_entries', 'battery_df', 'location_entries', 'filtered_location_entries', 'location_df', 'filtered_location_df', 'motion_activity_entries', 'motion_activity_df', 'transition_entries', 'transition_df'])
         fixed:ACCURACY_CONTROL_0 HAHFDC v/s HAMFDC accuracy_control_0 2
Battery starts at 99, ends at 76, drain = 23
         dict_keys(['trip_id', 'trip_id_base', 'trip_run', 'start_ts', 'end_ts', 'duration', 'eval_common_trip_id', 'eval_role', 'eval_role_base', 'eval_role

In [20]:
battery_drain_df.head()

,phone_os,phone_label,timeline,run,duration,role,battery_drain
0,android,ucb-sdb-android-1,unimodal_trip_car_bike_mtv_la,0,8078.722,accuracy_control,23.0
1,android,ucb-sdb-android-1,unimodal_trip_car_bike_mtv_la,1,8928.605,accuracy_control,25.0
2,android,ucb-sdb-android-1,unimodal_trip_car_bike_mtv_la,2,8850.759,accuracy_control,22.0
3,android,ucb-sdb-android-1,unimodal_trip_car_bike_mtv_la,3,7975.062,accuracy_control,16.0
4,android,ucb-sdb-android-1,unimodal_trip_car_bike_mtv_la,4,8224.197,accuracy_control,29.0


In [21]:
r2q_map = {"power_control": 0, "HAMFDC": 1, "MAHFDC": 2, "HAHFDC": 3, "accuracy_control": 4}
# right now, only the san jose data has the full comparison
q2r_complete_list = ["power", "HAMFDC", "MAHFDC", "HAHFDC", "accuracy"]
# others only have android or ios
q2r_android_list = ["power", "HAMFDC", "HAHFDC", "accuracy"]
q2r_ios_list = ["power", "MAHFDC", "HAHFDC", "accuracy"]

In [22]:
# Make a number so that can get the plots to come out in order
battery_drain_df["quality"] = battery_drain_df.role.apply(lambda r: r2q_map[r])

KeyError: 'MAMFDC'

In [ ]:
battery_drain_df.query("role == 'MAHFDC'").head()

## Displaying various groupings using boxplots

In [ ]:
ifig, ax_array = plt.subplots(nrows=2,ncols=3,figsize=(12,6), sharex=False, sharey=True)
timeline_list = ["train_bus_ebike_mtv_ucb", "car_scooter_brex_san_jose", "unimodal_trip_car_bike_mtv_la"]
for i, tl in enumerate(timeline_list):
    battery_drain_df.query("timeline == @tl & phone_os == 'android'").boxplot(ax = ax_array[0][i], column=["battery_drain"], by=["quality"], showbox=False, whis="range")
    ax_array[0][i].set_title(tl)
    battery_drain_df.query("timeline == @tl & phone_os == 'ios'").boxplot(ax = ax_array[1][i], column=["battery_drain"], by=["quality"], showbox=False, whis="range")
    ax_array[1][i].set_title("")

for i, ax in enumerate(ax_array[0]):
    if i == 1:
        ax.set_xticklabels(q2r_complete_list)
    else:
        ax.set_xticklabels(q2r_android_list)
    ax.set_xlabel("")

for i, ax in enumerate(ax_array[1]):
    if i == 1:
        ax.set_xticklabels(q2r_complete_list)
    else:
        ax.set_xticklabels(q2r_ios_list)
    ax.set_xlabel("")

ax_array[0][0].set_ylabel("Battery drain (android)")
ax_array[1][0].set_ylabel("Battery drain (iOS)")
ifig.suptitle("Power v/s quality over multiple timelines")
# ifig.tight_layout()

In [ ]:
battery_drain_df.query("quality == 1 & phone_os == 'ios' & timeline == 'car_scooter_brex_san_jose'").iloc[1:].describe()

In [ ]:
battery_drain_df.query("quality == 0 & phone_os == 'ios' & timeline == 'car_scooter_brex_san_jose'").iloc[1:].describe()

In [ ]:
battery_drain_df.query("quality == 2 & phone_os == 'ios' & timeline == 'car_scooter_brex_san_jose'").iloc[1:].describe()